<a href="https://colab.research.google.com/github/faiz-x-it/AiManga/blob/main/naifu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Berdasarkan versi modifikasi 4chan dari NovelAILeaks (naifu). sumber

Menggunakan front-end resmi + back-end yang dioptimalkan, dapat menembus batas 75 dan mendukung semua model.

Kredit: https://t.me/StableDiffusion_CN https://t.me/exlolicon

Terima kasih: Anonim, Redface, Penyihir Tembaga, Kagurazaka Sanae️, Jonathan, Goo Goo, Neko dan Inverter, Gaein nidb, 

In [ ]:
#@title ###NovelAILeaks API Backend (4chan Ver.) 
#@markdown Periksa status kerja GPU

!nvidia-smi
#@markdown ### 1. Unduh backend dan model Novel AI API
#@markdown Jika kecepatan unduh terlalu lambat, coba mulai ulang

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar 
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

#@markdown ### 2. Instal dependensi
#@markdown Jika kecepatan unduh terlalu lambat, coba mulai ulang

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared
!/content/naifu/venv/bin/python -m pip install -qq pytorch_lightning==1.7.7

#@markdown ### 3. Jalankan model
#@markdown Akses alamat pemetaan output (diakhiri dengan `trycloudflare.com` / `bore.pub`).
#@markdown - Harap tunggu hingga model dimuat (`Application startup complete` muncul) sebelum mengakses
#@markdown - Layanan yang disediakan oleh cloudflare terkadang memiliki waktu tunggu permintaan, dan bore tunnel dapat digunakan sebagai gantinya

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969 

#@markdown ### 4. (Opsional) Jalankan dengan model `animefull-latest` 7G
#@markdown menggunakan model animefull-final-pruned ukuran 4G secara default. Jika Anda ingin menggunakan model 7G animefull-terbaru, jalankan ini

%cd /content/
!mkdir /content/naifu/models/animefull-latest
!tar xf animefull-latest.tar -C /content/naifu/models/animefull-latest && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969 